# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [57]:
import os
import requests, json
from dotenv import load_dotenv
import re
import zipfile
import io
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
load_dotenv()

True

In [64]:
API_KEY = os.getenv('OPENROUTER_API_KEY')

MODEL = "phind/phind-codellama-34b"
# MODEL = "meta-llama/llama-3-70b-instruct"

# Setting up LLM

In [51]:
llm = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL)

In [52]:
def get_zip_structure(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_structure = io.StringIO()
        zip_ref.printdir(file=zip_structure)
        structure_text = zip_structure.getvalue().strip()
    return structure_text

In [67]:
INSTRUCT_WRAP_CODE = "\nWrap your code in ```python\n{CODE}\n```"

In [ ]:
input = "combine the images in this zip into a pdf file with one image as each page."

# input = "Convert the images in this zip file to a pdf by adding all images contained within to the PDF. Make sure to pay attention to the structure inside the zip file to make sure all code correctly handles the zipfile."

file = "images.zip"

# Generate detailed prompt from simple user prompt
gen_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful prompting assistant.\
    You must write instructions for creating a python script for completing the below user prompt.\
    Your job is to expand on the instructions below to make sure there is a defined plan of execution."""),
    ("user", "User Prompt: {input}\nFile: {file}\nFile Contents: {contents}")
])

prompt_code = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Write code based on the instructions with redundancy for errors."),
    ("user", "Instructions: {input}\nFile: {file}\nFile Contents: {contents}")
])

file_contents = "\nContents of unzipped file:\n" + get_zip_structure(file)

prompt_chain = gen_prompt | llm | output_parser

code_chain = prompt_code | llm | output_parser

prompt_output = prompt_chain.invoke({"input": input, "file": file, "contents": file_contents})

prompt_output += INSTRUCT_WRAP_CODE

code_output = code_chain.invoke({"input": prompt_output, "file": file, "contents": file_contents})

python_code = re.search(r'```python(?s:(.*?))```', code_output).group(1)

print(python_code)

In [78]:
print(prompt_output)
print(code_output)

Here is a plan of execution with an outline to combine the images in the zip file into a PDF file with one image per page:

**Plan of Execution:**

**Step 1: Unzip the File**

* Unzip the provided `images.zip` file to a directory on your local machine.
* This will extract the contents of the zip file, including the four image files: `seas.jpeg`, `platform.jpg`, `lulz.jpg`, and `picture.jpg`.

**Step 2: Install Required Software (if necessary)**

* If you don't already have a PDF creation tool installed, you may need to download and install one. Some popular options include:
	+ Adobe Acrobat (paid)
	+ PDFCreator (free)
	+ Img2PDF (free, online tool)

**Step 3: Combine Images into a PDF**

* Open your chosen PDF creation tool.
* Select the option to create a new PDF from images.
* Navigate to the unzipped image files and select all four images: `seas.jpeg`, `platform.jpg`, `lulz.jpg`, and `picture.jpg`.
* Choose the option to add each image as a separate page in the PDF.
* Set any desire

In [71]:
import zipfile
import img2pdf
import os

# Open the zip file
with zipfile.ZipFile('images.zip', 'r') as zip_file:
    # Extract the image files from the zip file
    image_files = [file for file in zip_file.namelist() if file.endswith(('.jpg', '.jpeg'))]
    
    # Create a new PDF file
    pdf_bytes = bytearray()
    
    # Add each image to the PDF file
    for image_file in image_files:
        # Extract the image file from the zip file
        with zip_file.open(image_file) as image:
            # Convert the image to a PDF page
            pdf_page = img2pdf.convert(image.read())
            # Add the PDF page to the PDF file
            pdf_bytes.extend(pdf_page)
    
    # Save the PDF file
    with open('images.pdf', 'wb') as pdf_file:
        pdf_file.write(pdf_bytes)

ImageOpenError: cannot read input image (not jpeg2000). PIL: error reading image: cannot identify image file <_io.BytesIO object at 0x119b1e2a0>